# Исследование поведения пользователей в приложении

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px
from plotly import graph_objects as go
from scipy import stats as st
import numpy as np
import math as mth
from scipy.stats import norm


## Описание проекта <a id="intro"></a>

В этом проекте будет исследоваться поведение пользователей приложения, в котором продаются продукты питания.


Узнаем, как пользователи доходят до покупки. Сколько пользователей доходит до покупки, а сколько — «застревает» на предыдущих шагах? На каких именно?


После этого исследуем результаты A/A/B-эксперимента. Дизайнеры захотели поменять шрифты во всём приложении, а менеджеры испугались, что пользователям будет непривычно. Договорились принять решение по результатам A/A/B-теста. Пользователей разбили на 3 группы: 2 контрольные со старыми шрифтами и одну экспериментальную — с новыми. Выясним, какой шрифт лучше.

###  Содержание

1. [Открытие данных, общая информация](#start)
2. [Предобработка данных](#preprocessing)
3. [Проверка данных](#test_data)
4. [Анализ воронка событий](#event_cart)
5. [Обработка результатов экспериментов](#processing_results)


6. [Общий вывод](#general_conclusion)

## Открытие данных, общая информация <a id="start"></a>

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/logs_exp.csv', sep="\t")

Каждая запись в логе — это действие пользователя, или событие.

    EventName — название события;
    DeviceIDHash **— уникальный идентификатор пользователя;
    EventTimestamp — время события;
    ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.

In [3]:
df.head()

,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
EventName         244126 non-null object
DeviceIDHash      244126 non-null int64
EventTimestamp    244126 non-null int64
ExpId             244126 non-null int64
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


In [5]:
df.duplicated().sum()

413

In [6]:
df.isna().sum()

EventName         0
DeviceIDHash      0
EventTimestamp    0
ExpId             0
dtype: int64

In [7]:
df.isnull().sum()

EventName         0
DeviceIDHash      0
EventTimestamp    0
ExpId             0
dtype: int64

### Вывод
В предобработке нужно поменять названия столбцов и удалить дубликаты. Из timestamp, потребуется достать нужные элементы даты.

## Предобработка данных <a id="preprocessing"></a>

In [8]:
#Удадяем дубликаты
df = df.drop_duplicates()

#Переименовываем столбцы
df.columns = ['event_name', 'user_id', 'event_timestamp', 'exp_id']

#Достаём дату/время из таймстамп
df['event_datetime'] = pd.to_datetime(df['event_timestamp'], unit='s')

#Достаём дату
df['event_date'] = df['event_datetime'].dt.date

#Переводим строку в нужный формат
df['event_date'] = pd.to_datetime(df['event_date'], format='%Y-%m-%d')

In [9]:
#Переводим строку в нужный формат
df['event_datetime'] = pd.to_datetime(df['event_datetime'], format='%Y.%m.%d')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243713 entries, 0 to 244125
Data columns (total 6 columns):
event_name         243713 non-null object
user_id            243713 non-null int64
event_timestamp    243713 non-null int64
exp_id             243713 non-null int64
event_datetime     243713 non-null datetime64[ns]
event_date         243713 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(1)
memory usage: 13.0+ MB


In [11]:
df.head()

,event_name,user_id,event_timestamp,exp_id,event_datetime,event_date
0,MainScreenAppear,4575588528974610257,1564029816,246,2019-07-25 04:43:36,2019-07-25
1,MainScreenAppear,7416695313311560658,1564053102,246,2019-07-25 11:11:42,2019-07-25
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248,2019-07-25 11:28:47,2019-07-25
3,CartScreenAppear,3518123091307005509,1564054127,248,2019-07-25 11:28:47,2019-07-25
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248,2019-07-25 11:48:42,2019-07-25


## Проверка данных <a id="test_data"></a>

In [12]:
print('В таблице у нас есть', df['event_name'].nunique(), 'уникальных событий')
print()
for i in df['event_name'].unique():
    print(i)
print()
print()
print('Всего в таблице', df['user_id'].nunique(), 'пользователей с уникальным user_id')

В таблице у нас есть 5 уникальных событий

MainScreenAppear
PaymentScreenSuccessful
CartScreenAppear
OffersScreenAppear
Tutorial


Всего в таблице 7551 пользователей с уникальным user_id


In [13]:
print('Медианное количество событий на одного пользователя: ', df.groupby('user_id').agg({'event_name':'count'}).median()[0])

Медианное количество событий на одного пользователя:  20.0


In [14]:
print('Минимальная дата:', df.event_date.min())
print('Максимальная дата:', df.event_date.max())

Минимальная дата: 2019-07-25 00:00:00
Максимальная дата: 2019-08-07 00:00:00


In [15]:
df.groupby('event_date').agg('count')[['user_id']].reset_index()

,event_date,user_id
0,2019-07-25,9
1,2019-07-26,31
2,2019-07-27,55
3,2019-07-28,105
4,2019-07-29,184
5,2019-07-30,412
6,2019-07-31,2030
7,2019-08-01,36141
8,2019-08-02,35554
9,2019-08-03,33282


In [31]:
# Посмотрим на полноту данных
fig = px.bar(df.groupby('event_date').agg('count')[['user_id']].reset_index(),
             x="event_date",
             y="user_id")
fig.show()

До 31 июля 2019 года, данных мало. Отбросим данные с датой раньше 1 августа.  В итоге у нас останутся данные за 8 дней 

In [32]:
df = df.query('event_date > "2019-07-31"')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240887 entries, 2828 to 244125
Data columns (total 6 columns):
event_name         240887 non-null object
user_id            240887 non-null int64
event_timestamp    240887 non-null int64
exp_id             240887 non-null int64
event_datetime     240887 non-null datetime64[ns]
event_date         240887 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(1)
memory usage: 12.9+ MB


In [33]:
# Проверим, что у нас есть пользователи из всех трёх экспериментальных групп
df.groupby('exp_id').agg('count')[['user_id']]

,user_id
exp_id,
246,79302
247,77022
248,84563


### Вывод

В таблице у нас есть 5 уникальных событий

    MainScreenAppear
    PaymentScreenSuccessful
    CartScreenAppear
    OffersScreenAppear
    Tutorial

Всего в исходной таблице 7551 пользователей с уникальным user_id.
Медианное количество событий на одного пользователя:  20
Минимальная дата: 2019-07-25
Максимальная дата: 2019-08-07

До 1 августа 2019 года, данных мало. Отбросили данные с датой раньше 1 августа. Теряем около 3 000 событий из 243 713 – это 1,2%. В итоге у нас остались данные за 7 дней.
	
Группа 246 	67741
Группа 247 	64716
Группа 248 	72289

В датафрейме остались данные о всех 3-х эксперементальных группах. Количество событий в группе 248 больше, чем 247 и 246.

## Анализ воронка событий <a id="event_cart"></a>

In [34]:
# Посмотрим, какие события есть в логах, как часто они встречаются
df.groupby('event_name') \
                    .agg('count')[['user_id']].reset_index().sort_values(by='user_id', ascending=False)

,event_name,user_id
1,MainScreenAppear,117328
2,OffersScreenAppear,46333
0,CartScreenAppear,42303
3,PaymentScreenSuccessful,33918
4,Tutorial,1005


У нас есть 5 событий, предпологаю, что пользователь должен двигаться по этапам вот в таком порядке:
    
    MainScreenAppear - открыт главный экран приложения
    OffersScreenAppear - открыт экран с предложениями
    CartScreenAppear - открыта корзина
    PaymentScreenSuccessful - оплата
    Tutorial – открыт туториал

In [35]:
#Для пользователей посчитаем время первого срабатывания каждого события
users = df.pivot_table(
                        index='user_id', 
                        columns='event_name', 
                        values='event_datetime',
                        aggfunc='min'
                        ).reset_index()

# Данные для построения продуктовой воронки без учёта последовательности событий
main_screen = users[~users['MainScreenAppear'].isna()].shape[0]
offers_screen = users[~users['OffersScreenAppear'].isna()].shape[0]
cart_screen = users[~users['CartScreenAppear'].isna()].shape[0]
payment_screen = users[~users['PaymentScreenSuccessful'].isna()].shape[0]

# Продуктовая воронка с учётом последовательности событий
step_1 = ~users['MainScreenAppear'].isna()
step_2 = step_1 & (users['OffersScreenAppear'] > users['MainScreenAppear'])
step_3 = step_2 & (users['CartScreenAppear'] > users['OffersScreenAppear'])
step_4 = step_3 & (users['PaymentScreenSuccessful'] > users['CartScreenAppear'])

n_main_screen = users[step_1].shape[0]
n_offers_screen = users[step_2].shape[0]
n_cart_screen = users[step_3].shape[0]
n_payment_screen = users[step_4].shape[0]

In [36]:
#Создадим переменную с названиями событий на русском
status_rus = ['Открыли главный экран приложения',
'Открыли экран с предложениями',
'Открыли корзину',
'Оплатили']


#Построим воронку БЕЗ учёта последовательности событий
fig = go.Figure(go.Funnel(
    y = status_rus,
    x = [main_screen, offers_screen, cart_screen, payment_screen]
    ))
fig.update_layout(title='Воронка БЕЗ УЧЁТА последовательности событий')
fig.show()

    Доля пользователей которые открыли главный экран приложения – 38,5% от общего числа пользователей
    Доля пользователей которые открыли экран с предложениями – 23,8% от общего числа пользователей
    Доля пользователей которые открыли корзину – 19,4% от общего числа пользователей
    Доля пользователей которые оплатили –18,4% от общего числа пользователей

In [22]:
#Посмотрим сколько пользователей не совершили не одного события, кроме открытия главного экрана
users[users['CartScreenAppear'].isna()   
      & users['OffersScreenAppear'].isna()
      & users['PaymentScreenSuccessful'].isna()]

event_name,user_id,CartScreenAppear,MainScreenAppear,OffersScreenAppear,PaymentScreenSuccessful,Tutorial
0,6888746892508752,NaT,2019-08-06 14:06:34,NaT,NaT,NaT
3,7435777799948366,NaT,2019-08-05 08:06:34,NaT,NaT,NaT
11,20449203507642281,NaT,2019-08-06 21:04:21,NaT,NaT,NaT
13,24144092107925848,NaT,2019-08-01 12:53:16,NaT,NaT,NaT
14,26317307137967461,NaT,2019-08-01 14:33:57,NaT,NaT,NaT
...,...,...,...,...,...,...
7523,9212523802225607780,NaT,2019-08-02 06:06:51,NaT,NaT,2019-08-02 06:04:56
7526,9215717049765076788,NaT,2019-08-01 11:39:57,NaT,NaT,NaT
7527,9216094175241772920,NaT,2019-08-02 15:23:52,NaT,NaT,NaT
7532,9221926045299980007,NaT,2019-08-01 17:30:27,NaT,NaT,NaT


2856 пользователей из 7286 вообще не совершали никаких действий, они только открывали главный экран.

In [23]:
#Построим воронку с учётом последовательности событий
fig = go.Figure(go.Funnel(
    y = status_rus,
    x = [n_main_screen, n_offers_screen, n_cart_screen, n_payment_screen]
    ))
fig.update_layout(title='Воронка С УЧЁТОМ последовательности событий')
fig.show()

Количество пользователей на первом шаге, конечно, совпадает, а дальше начинаются расхождения. Это значит, что не все пользователи идут по ожидаемому пути. 

### Вывод

У нас есть 5 событий, предполагаю, что пользователь должен двигаться по этапам вот в таком порядке:

    1 - MainScreenAppear - открыт главный экран приложения
    2 - OffersScreenAppear - открыт экран с предложениями
    3 - CartScreenAppear - открыта корзина
    4 - PaymentScreenSuccessful - оплата

Если построить воронку без учёта последовательности, то конверсия в покупку составляет 47,5%. Больше всего пользователей в такой воронке теряется на этапе перехода от главного экрана на экран с офферами. Там мы теряем около 39% пользователей.

В воронке с учётом логической последовательности, конверсия меньше 5,6%. В этой воронке больше всего пользователей теряется на этапе перехода из корзины в оплату. Потеря примерно 75%.

Такая разница в конверсиях, говорит о том, что большинство пользователей идут не по ожидаемому пути.

2856 пользователей из 7286 вообще не совершали никаких действий, они только открывали главный экран.


## Обработка результатов экспериментов <a id="processing_results"></a>

In [26]:
#Напишем функцию для z-теста
def z_test(n1, x1, n2, x2, alpha):
    p1 = x1 / n1
    p2 = x2 / n2
    p = (p1 * n1 + p2 * n2) / (n1 + n2)
    SE = np.sqrt(p*(1-p)*(1/n1+1/n2))
    m = SE*norm.ppf(1-alpha/2)
    z_stat = (p1-p2)/SE
    p=norm.cdf(np.abs(z_stat))
    return 2 * (1-p)

#### Сравнение результатов А/А теста

In [27]:
events_AA = df.copy()
events_AA['exp_id'] = events_AA['exp_id'].astype(str).str.replace('246',
                                                                    'A1').str.replace('247',
                                                                                      'A2').str.replace('248','B')
#количество уников
totals_AA = events_AA.groupby('exp_id').agg({'user_id':'nunique'})['user_id']
counts_AA = events_AA.pivot_table(index='event_name', 
                                  columns = 'exp_id', 
                                  values='user_id', aggfunc='nunique').reset_index()
counts_AA = counts_AA[['event_name','A1','A2']]

# Коррекция уровня значимости(по 5 гипотез в АА и АВ тестах)                                                                                                        
counts_AA['alpha'] = 0.05 / (counts_AA.shape[0]*2)

In [28]:
#Выполняем тест
counts_AA['p_value'] = counts_AA.apply(lambda x: z_test(totals_AA['A1'],
                                                  x['A1'], 
                                                  totals_AA['A2'],
                                                  x['A2'],
                                                  counts_AA['alpha']), axis=1)
counts_AA

exp_id,event_name,A1,A2,alpha,p_value
0,CartScreenAppear,1266,1238,0.005,0.228834
1,MainScreenAppear,2450,2476,0.005,0.757060
2,OffersScreenAppear,1542,1520,0.005,0.248095
3,PaymentScreenSuccessful,1200,1158,0.005,0.114567
4,Tutorial,278,283,0.005,0.937700


Результаты теста показали, что не удалось обнаружить статистически значимой разници между группами А1 и А2, ни по одному из событий. Это и к лучшему, ведь проверяли А/А тест.

#### Сравнение результатов А/B теста

In [29]:
#считаем, что  246 и 247 эквивалентны
events = df.copy()
events['exp_id'] = events['exp_id'].astype(str).str.replace('246|247', 'A').str.replace('248', 'B')

#количество уников
totals = events.groupby('exp_id').agg({'user_id':'nunique'})['user_id']
counts = events.pivot_table(index='event_name', columns = 'exp_id', values='user_id', aggfunc='nunique')

# Коррекция уровня значимости(по 5 гипотез в АА и АВ тестах)
counts['alpha'] = 0.05 / (counts.shape[0]*2)


In [30]:
counts['p_value'] = counts.apply(lambda x: z_test(totals['A'],
                                                  x['A'], 
                                                  totals['B'],
                                                  x['B'],
                                                  counts['alpha']), axis=1)
counts

exp_id,A,B,alpha,p_value
event_name,,,,
CartScreenAppear,2504,1230,0.005,0.181759
MainScreenAppear,4926,2493,0.005,0.294245
OffersScreenAppear,3062,1531,0.005,0.434255
PaymentScreenSuccessful,2358,1181,0.005,0.600429
Tutorial,561,279,0.005,0.764862


## Общий вывод <a id="general_conclusion"></a>

### Анализ воронки
У нас есть 5 событий, предполагаю, что пользователь должен двигаться по этапам вот в таком порядке:

    1 - MainScreenAppear - открыт главный экран приложения
    2 - OffersScreenAppear - открыт экран с предложениями
    3 - CartScreenAppear - открыта корзина
    4 - PaymentScreenSuccessful - оплата

Если построить воронку без учёта последовательности, то конверсия в покупку составляет 47,5%. Больше всего пользователей в такой воронке теряется на этапе перехода от главного экрана на экран с офферами. Там мы теряем около 39% пользователей.

В воронке с учётом логической последовательности, конверсия меньше 5,6%. В этой воронке больше всего пользователей теряется на этапе перехода из корзины в оплату. Потеря примерно 75%.

Такая разница в конверсиях, говорит о том, что большинство пользователей идут не по ожидаемому пути.

2856 пользователей из 7286 вообще не совершали никаких действий, они только открывали главный экран.

### Анализ результатов экспериментов

По результатам А/А теста с учётом поправки Бенферрони, не получилось найти значимых различий ни по одному событию в экспериментальных группах 246 и 247, это хорошо, ведь это А/А – тест.

По результатам  A/B теста, аналогично не удалось увидеть статистически значимых различий между группами, ни по одному из событий. 

Можно сделать вывод, что изменение шрифтов никак не влияет на конверсию. 
